환경변수 가져오기

In [1]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

## 메인 로직
사용자로부터 시나리오 프롬프트를 파라미터로 받아야 함.
그리고 빌드 번호도 받아야 함

In [8]:
import os
import json
import base64
from datetime import datetime
from typing import Optional, List, Any

from pydantic import BaseModel
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.runnables import Runnable
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import ToolMessage

# 모델 정의
model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0, max_tokens=4000)

# MCP 클라이언트
client = MultiServerMCPClient({
    "playwright": {
        "url": "http://localhost:8005/sse",
        "transport": "sse",
    }
})

# 출력 디렉토리 생성
BASE_DIR = "./results"
build_num = 1   # 임시로 빌드 number가 1
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
test_id = f"{timestamp}_report_{build_num}"
output_dir = os.path.join(BASE_DIR, test_id)
os.makedirs(output_dir, exist_ok=True)

# Output Parser
class FailedStep(BaseModel):
    num: int
    message: str

class WebTestResult(BaseModel):
    status: bool
    duration: Optional[float]
    feedback: str
    fail: Optional[List[FailedStep]]

parser = PydanticOutputParser(pydantic_object=WebTestResult)

summary_prompt = PromptTemplate(
    template="""
다음 웹 자동화 테스트 로그를 바탕으로 결과를 아래 형식의 JSON으로 요약해줘:

- status: 테스트 성공 여부 (true/false)
- duration: 테스트 소요 시간 (초 단위, 숫자)
- feedback: 전체 테스트에 대한 요약 피드백
- fail: 실패한 동작이 있을 경우,
  - 실패한 "스텝 번호" (입력 시나리오 steps에서 몇 번째 step인지, 예: 2번 step이면 num: 2),
  - 실패 이유를 message에 담아 리스트로 작성할 것

⚠️ 주의:
- ‘실패’ 또는 ‘의도한 행동과 다른 결과’가 있었다면 반드시 fail 항목을 채워야 합니다.
- "실패한 것 같지만 일단 성공"은 허용되지 않습니다.
- 예를 들어 클릭이 다른 대상에 잘못 적용되면 무조건 실패로 간주하고 fail 리스트에 포함하세요.

{format_instructions}

로그:
{raw_result}
""",
    input_variables=["raw_result"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)
summary_chain: Runnable = summary_prompt | model | parser

DEFAULT_RESULT_STEP = "성공 여부, 속도, 피드백을 포함해 결과를 요약한다."

# 시나리오 단위 실행
async def run_test():
    await client.__aenter__()
    tools = client.get_tools()

    scenarios = [
        # {
        #     "title": "네이버 검색 기능 확인",
        #     "steps": [
        #         "https://naver.com 에 접속한다.",
        #         "검색창에 SSAFY를 입력하고 검색버튼을 누른다.",
        #         "SSAFY와 관련된 게시물이 나오는지 확인한다."
        #     ]
        # },
        {
            "title": "유튜브 나중에 볼 동영상 여부 테스트",
            "steps": [
                "https://www.youtube.com 에 접속한다",
                "왼쪽의 네비게이션 바에서 '김정우'를 클릭한다.",
                "스크린샷을 찍는다"
            ]
        },
        # {
        #     "title": "유튜브 영상 검색 테스트",
        #     "steps": [
        #         "https://www.youtube.com 에 접속한다.",
        #         "'lofi hip hop'을 검색창에 입력하고 Enter를 누른다.",
        #         "스크린샷을 찍어줘"
        #     ]
        # },
        # {
        #     "title": "라프텔 화면 캡쳐 테스트",
        #     "steps": [
        #         "https://laftel.net/ 에 접속한다.",
        #         "접속 화면을 스크린샷 찍어줘",
        #         "상단 탭에서 '요일별 신작'을 클릭해",
        #         "화면에서 '수요일'부분 아래에 있는 첫번째 이미지를 클릭해",
        #         "화면을 스크린샷 찍어줘"
        #     ]
        # }
    ]

    for i, scenario in enumerate(scenarios, 1):
        agent = create_react_agent(model, tools)
        await run_scenario(agent, scenario, i)

    print("모든 테스트 완료:", output_dir)

# 시나리오 실행
async def run_scenario(agent, scenario: dict, index: int):
    scenario_dir = os.path.join(output_dir, str(index))
    screenshot_dir = os.path.join(scenario_dir, "screenshots")
    os.makedirs(screenshot_dir, exist_ok=True)

    screenshots, log_text = await run_with_callback(agent, scenario["steps"], screenshot_dir)
    summary = await summarize_result(log_text)
    save_result(scenario, summary, screenshots, scenario_dir)

# MCP 실행 + 콜백 수집
async def run_with_callback(agent, steps: list[str], screenshot_dir: str):
    collected_text_chunks = []
    screenshot_files = []

    async def callback(event: dict[str, Any]):
        content = event["content"]

        # 스크린샷은 ToolMessage로, artifact 안에 들어있는 base64 image
        if isinstance(content, ToolMessage) and getattr(content, "artifact", None):
            for artifact in content.artifact:
                if getattr(artifact, "type", "") == "image" and hasattr(artifact, "data"):
                    filename = f"{len(screenshot_files)+1}.png"
                    filepath = os.path.join(screenshot_dir, filename)
                    with open(filepath, "wb") as f:
                        f.write(base64.b64decode(artifact.data))
                    screenshot_files.append(filename)
                    print(f"📸 스크린샷 저장됨: {filename}")
        else:
            text = content.content if hasattr(content, "content") else str(content)
            collected_text_chunks.append(text)

    from utils import astream_graph
    instruction = "\n".join(f"{i+1}. {s}" for i, s in enumerate(steps + [DEFAULT_RESULT_STEP]))
    await astream_graph(agent, inputs={"messages": instruction}, stream_mode="messages", callback=callback)
    return screenshot_files, "\n".join(collected_text_chunks)

# 요약 처리
async def summarize_result(raw_text: str) -> WebTestResult:
    return await summary_chain.ainvoke({"raw_result": raw_text})

# 결과 저장
def save_result(scenario: dict, result: WebTestResult, screenshots: List[str], scenario_dir: str):
    result_json = {
        "title": scenario["title"],
        "status": result.status,
        "duration": result.duration,
        "feedback": result.feedback,
        "fail": [f.model_dump() for f in result.fail] if result.fail else None,
        "screenshots": screenshots
    }
    with open(os.path.join(scenario_dir, "result.json"), "w", encoding="utf-8") as f:
        json.dump(result_json, f, ensure_ascii=False, indent=2)

# 실행
await run_test()


📸 스크린샷 저장됨: 1.png
모든 테스트 완료: ./results\20250423-111510_report_1


Error in sse_reader: 


그래프를 실행하여 결과를 확인합니다.